In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col,asc

In [2]:
spark=SparkSession.builder.appName("sensors_maximum_value_PM10").getOrCreate()

25/10/13 17:30:51 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
df=spark.read.text("data.txt")
df.show()

+------------------+
|             value|
+------------------+
|s1,2016-01-01,20.5|
|s2,2016-01-01,30.1|
|s1,2016-01-02,60.2|
|s2,2016-01-02,20.4|
|s1,2016-01-03,60.2|
|s2,2016-01-03,52.5|
+------------------+



In [4]:
df = df.withColumn("id", split(col("value"), ",")[0]) \
             .withColumn("date", split(col("value"), ",")[1]) \
             .withColumn("reading", split(col("value"), ",")[2]) \
             .drop("value")

df.show()

+---+----------+-------+
| id|      date|reading|
+---+----------+-------+
| s1|2016-01-01|   20.5|
| s2|2016-01-01|   30.1|
| s1|2016-01-02|   60.2|
| s2|2016-01-02|   20.4|
| s1|2016-01-03|   60.2|
| s2|2016-01-03|   52.5|
+---+----------+-------+



In [5]:
df=df.orderBy(asc("reading"))
df.show()

+---+----------+-------+
| id|      date|reading|
+---+----------+-------+
| s2|2016-01-02|   20.4|
| s1|2016-01-01|   20.5|
| s2|2016-01-01|   30.1|
| s2|2016-01-03|   52.5|
| s1|2016-01-02|   60.2|
| s1|2016-01-03|   60.2|
+---+----------+-------+



In [6]:
reading=df.select("reading")
reading.show()

+-------+
|reading|
+-------+
|   20.4|
|   20.5|
|   30.1|
|   52.5|
|   60.2|
|   60.2|
+-------+



In [10]:
top_3=spark.createDataFrame(reading.take(3))
top_3.show()

+-------+
|reading|
+-------+
|   20.4|
|   20.5|
|   30.1|
+-------+



In [11]:
top_3.write.mode("overwrite").csv("./output", header=True)